# UserStream を使わないでタイムラインをリアルタイム取得

In [6]:
# Python 3.6.2 :: Anaconda, Inc.

import requests_oauthlib
import requests
import datetime
import ast
import json
import datetime
import time

In [7]:
CK = "XXX"
CS = "XXX"
AT = "XXX"
AS = "XXX"

auth = requests_oauthlib.OAuth1(CK, CS, AT, AS)

In [8]:
# twitter の API 叩いてる関係の関数
# return req されてるやつは、raw data が返ってきてるので、その後よろしくやる

# フォロワーを取得
def get_followers():
    url = "https://api.twitter.com/1.1/followers/list.json"
    req = requests.get(url, auth=auth)
    
    if req.status_code != 200:
        return dict()
    
    raw_data = json.loads(req.text)
    followers = raw_data["users"]
    res = {}
    for f in followers:
        res[f["screen_name"]] = f["id"]
    return res


# images: [open("/path/to/image.png", "rb")], 複数可
# 画像ツイートは、一度画像ファイルを twitter サーバにアップロードして、
# 返ってきた media id を指定してツイートする
def get_media_ids(images):
    url = "https://upload.twitter.com/1.1/media/upload.json"
    
    media_ids = []
    for im in images:
        file = {"media": im}
        req = requests.post(url, auth=auth, files=file)
    
        if req.status_code != 200:
            return []
        
        media_ids.append(str(json.loads(req.text)["media_id"]))
    return media_ids


# followers で指定したユーザたちのツイートを取得
def get_timeline(followers):
    data = {"follow": followers.values()}
    url = "https://stream.twitter.com/1.1/statuses/filter.json"
    req = requests.post(url, auth=auth, stream=True, data=data)

    return req
            
    
# tweet の ID からツイートの raw data を取得
def get_tweet_by_id(tw_id):
    url = "https://api.twitter.com/1.1/statuses/show.json?id="+str(tw_id)
    req = requests.get(url, auth=auth)
    
    return req
    

def post_tweet(text):
    url = "https://api.twitter.com/1.1/statuses/update.json"
    data = {"status": text+str(datetime.datetime.now())}
    req = requests.post(url, auth=auth, data=data)

    return req
    
# tweet ID を指定して、そのツイートへリプライ
# user はそのツイートをしたユーザの ID
def post_tweet_reply(user, tweet_id, text):
    url = "https://api.twitter.com/1.1/statuses/update.json"
    data = {"status": "@"+user+" "+text, "in_reply_to_status_id": tweet_id}
    req = requests.post(url, auth=auth, data=data)
    
    return req
    
# 画像つきツイート
# media_ids はリストにして複数可 (4枚まで)
def post_tweet_with_media(media_ids, text):
    media_ids = ",".join(media_ids)
    url = "https://api.twitter.com/1.1/statuses/update.json"
    data = {"status": text, "media_ids": media_ids}
    req = requests.post(url, auth=auth, data=data)
    
    return req

# 画像つきでリプライ
# media_ids 以外は post_tweet_reply と同じ
def post_tweet_reply_with_media(media_ids, user, tweet_id, text):
    media_ids = ",".join(media_ids)
    url = "https://api.twitter.com/1.1/statuses/update.json"
    data = {"status": "@"+user+" "+text, "media_ids": media_ids, "in_reply_to_status_id": tweet_id}
    req = requests.post(url, auth=auth, data=data)
    
    return req


# user_id で指定したユーザのツイート count 個を取得
def get_user_timeline(user_id, count):
    url = "https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name="+user_id+"&count="+str(count)
    req = requests.get(url, auth=auth)
    
    if req.status_code != 200:
        return []
    
    return req


# タイムラインをストリームで表示
def show_timeline():
    while True:
        followers = get_followers()
        timeline = get_timeline(followers)

        # API 制限かかったり、なんかエラーが起きたらとりあえず待つ
        if timeline.status_code != 200:
            print("!")
            # time.sleep(60)

        t = datetime.datetime.now()
        print(t)

        # タイムラインにツイートが投稿されると、ここが回る
        # そうじゃないときも謎のエラーが出るので、except で pass する
        for line in timeline.iter_lines():
            try:
                # 15　分に一回 break して取得し直す
                # これ別にいらないかも
                if (datetime.datetime.now() - t).seconds > 900:
                    break

                stream = line.decode("utf-8")
                raw_data = json.loads(stream)

                print(raw_data["user"]["screen_name"], ":", raw_data["text"])

            except Exception as e:
                # print(e)
                pass

In [ ]:
# とりあえずタイムラインを垂れ流し続けるやつ
show_timeline()